In [1]:
!pip install lightgbm
!pip install xgboost



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tabulate



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tabulate import tabulate

# Define preprocessing function
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text
    return ""

# Load dataset
csv_file = r"C:\Users\HP\OneDrive\Desktop\new_pro\new_dataset.csv"
df = pd.read_csv(csv_file, low_memory=False)

# Apply text preprocessing
if 'description' in df.columns and 'label' in df.columns:
    df['text_preprocessed'] = df['description'].apply(preprocess_text)
else:
    raise ValueError("CSV file must contain 'description' and 'label' columns")

# Ensure no empty texts remain
df['text_preprocessed'].replace('', np.nan, inplace=True)
df.dropna(subset=['text_preprocessed'], inplace=True)

# Use CountVectorizer to create a Bag of Words (sparse matrix)
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['text_preprocessed'])

# Save bag of words to a file
vocab = vectorizer.get_feature_names_out()
with open("bag_of_words.txt", 'w') as file:
    for word in vocab:
        file.write(f'{word}\n')
print("Bag of words saved to 'bag_of_words.txt'")

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# Split data into train and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define and train Logistic Regression model
lr_model = LogisticRegression(C=1.0, solver='lbfgs', max_iter=1000)
lr_model.fit(X_train, y_train)

# Make predictions with Logistic Regression
y_pred_lr = lr_model.predict(X_test)

# Evaluate the Logistic Regression model
lr_accuracy = accuracy_score(y_test, y_pred_lr)
lr_report = classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_, output_dict=True)
lr_precision = lr_report['macro avg']['precision']
lr_recall = lr_report['macro avg']['recall']
lr_f1_score = lr_report['macro avg']['f1-score']

# Add Logistic Regression results to the table
results = [["Logistic Regression", lr_accuracy, lr_precision, lr_recall, lr_f1_score]]
headers = ["Model", "Accuracy", "Precision", "Recall", "F1 Score"]

# Display the final result in a clean table
print(tabulate(results, headers=headers, tablefmt="grid"))


C:\Users\HP\AppData\Local\Temp\ipykernel_13112\3134007845.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['text_preprocessed'].replace('', np.nan, inplace=True)


Bag of words saved to 'bag_of_words.txt'
+---------------------+------------+-------------+----------+------------+
| Model               |   Accuracy |   Precision |   Recall |   F1 Score |
+=====================+============+=============+==========+============+
| Logistic Regression |   0.905066 |    0.909291 | 0.870639 |   0.888213 |
+---------------------+------------+-------------+----------+------------+
